In [38]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [39]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CICIDS2017_df, CTU13_df = pd.concat(CICIDS2017_df_list, ignore_index=True), pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [40]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [41]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [42]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [43]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [44]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max',
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std',
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size',
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
])

In [45]:
# Train a Binary Random Forest model on CTU13 dataset
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])

start = time.time()
rfClf_bin_ctu13 = RandomForestClassifier(n_jobs=-2)
rfClf_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

# Save the binary Random Forest model trained on CTU13 dataset
with open('../Pickle Files/RFBinaryCTU13.pkl', 'wb') as file:
    pickle.dump(rfClf_bin_ctu13, file)

Training time (CTU13 Binary):  1.081782341003418


In [46]:
# Train a Binary Random Forest model on CICIDS2017 dataset
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

start = time.time()
rfClf_bin_cicids = RandomForestClassifier(n_jobs=-2)
rfClf_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

# Save the binary Random Forest model trained on CICIDS2017 dataset
with open('../Pickle Files/RFBinaryCICIDS2017.pkl', 'wb') as file:
    pickle.dump(rfClf_bin_cicids, file)

Training time (CICIDS2017 Binary):  95.09497356414795


In [47]:
# Train a Multiclass Random Forest model on CICIDS2017 dataset
start = time.time()
rfClf_multi_cicids = RandomForestClassifier(n_jobs=-2)
rfClf_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

# Save the multiclass Random Forest model trained on CICIDS2017 dataset
with open('../Pickle Files/RFMulticlassCICIDS2017.pkl', 'wb') as file:
    pickle.dump(rfClf_multi_cicids, file)

Training time (CICIDS2017 Multiclass):  91.9353301525116


In [48]:
# Load the binary Random Forest model trained on CTU13 dataset
with open('../Pickle Files/RFBinaryCTU13.pkl', 'rb') as file:
    rfClf_bin_ctu13 = pickle.load(file)
    
# Test on CTU13 dataset
predictions_bin_ctu13 = rfClf_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Acc (CTU13 Binary): 0.997370
F1-score (CTU13 Binary): 0.996883


Pred,Benign,Malicious
True,,
Benign,21275,51
Malicious,46,15513


In [49]:
# Load the binary Random Forest model trained on CICIDS2017 dataset
with open('../Pickle Files/RFBinaryCICIDS2017.pkl', 'rb') as file:
    rfClf_bin_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = rfClf_bin_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Binary): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary): 0.989724
F1-score (CICIDS2017 Binary): 0.974243


Pred,Benign,Malicious
True,,
Benign,899689,8839
Malicious,2785,219838


In [50]:
# Load the multiclass Random Forest model trained on CICIDS2017 dataset
with open('../Pickle Files/RFMulticlassCICIDS2017.pkl', 'rb') as file:
    rfClf_multi_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_multi_cicids = rfClf_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("F1-score (CICIDS2017 Multiclass): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='macro')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multiclass): 0.989405
F1-score (CICIDS2017 Multiclass): 0.815482


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,899651,78,64,23,8104,95,1,14,0,0,400,83,14,0,1
Bot,395,375,0,0,0,0,0,0,0,0,0,0,0,0,0
DDoS,75,0,51435,0,0,0,0,0,0,0,0,0,0,0,0
DoS GoldenEye,40,0,0,4097,1,2,0,0,0,0,0,0,0,0,0
DoS Hulk,2069,0,1,0,89533,0,0,0,0,0,3,0,0,0,0
DoS Slowhttptest,18,0,0,1,0,2131,7,0,0,0,0,0,0,0,0
DoS slowloris,5,0,0,0,1,2,2354,0,0,0,0,0,0,0,0
FTP-Patator,46,0,0,0,0,0,0,3148,0,0,0,0,0,0,1
Heartbleed,1,0,0,0,0,0,0,0,4,0,0,0,0,0,0


In [51]:
# Load the binary Random Forest model trained on CTU13 dataset
with open('../Pickle Files/RFBinaryCTU13.pkl', 'rb') as file:
    rfClf_bin_ctu13 = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = rfClf_bin_ctu13.predict(test_cicids[features])
print("Acc (CTU13 to CICIDS2017): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CTU13 to CICIDS2017): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CTU13 to CICIDS2017): 0.350702
F1-score (CTU13 to CICIDS2017): 0.298548


Pred,Benign,Malicious
True,,
Benign,240400,668128
Malicious,66326,156297


In [54]:
# Load the binary Random Forest model trained on CICIDS2017
with open('../Pickle Files/RFBinaryCICIDS2017.pkl', 'rb') as file:
    rfClf_bin = pickle.load(file)

# Test on CTU13 dataset
predictions_bin = rfClf_bin.predict(test_ctu13[features])
print("Acc (CICIDS2017 Binary to CTU13): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin)))
print("F1-score (CICIDS2017 Binary to CTU13): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary to CTU13): 0.578447
F1-score (CICIDS2017 Binary to CTU13): 0.001285


Pred,Benign,Malicious
True,,
Benign,21326,0
Malicious,15549,10


In [56]:
# Load the multiclass Random Forest model trained on CICIDS2017
with open('../Pickle Files/RFMulticlassCICIDS2017.pkl', 'rb') as file:
    rfClf_multi = pickle.load(file)

# Test on CTU13 dataset
predictions_multi = rfClf_multi.predict(test_ctu13[features])
print("Acc (CICIDS2017 Multi-class to CTU13): {:3f}".format(accuracy_score(test_ctu13[' Label'], predictions_multi)))
print("F1-score (CICIDS2017 Multi-class to CTU13): {:3f}".format(f1_score(test_ctu13[' Label'], predictions_multi, average='macro')))
pd.crosstab(test_ctu13[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multi-class to CTU13): 0.578175
F1-score (CICIDS2017 Multi-class to CTU13): 0.244267


Pred,BENIGN,DoS Hulk
True,,
BENIGN,21326,0
Bot,15552,7
